<a href="https://colab.research.google.com/github/lupis30puc/bipm_text_analytics_exercises/blob/main/Exercise_4_text_representation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
import pickle
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from google.colab import files
from scipy.spatial.distance import cosine, cdist
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import *

     |████████████████████████████████| 778kB 2.6MB/s 
     |████████████████████████████████| 890kB 7.0MB/s 
     |████████████████████████████████| 3.0MB 13.8MB/s 
     |████████████████████████████████| 1.1MB 31.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=1a98b680ae2f13de650a1021ee78ad5870c024a1f0f0db8ffa4b00df737956e6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
uploaded = files.upload()

Saving Stemmed.pkl to Stemmed.pkl


In [ ]:
# Load dataset and check first entry
stem_data = pickle.load(open('Stemmed.pkl','rb'))
print(stem_data[0])

car wonder enlighten car saw dai door sport car look late earli call bricklin door small addit bumper separ rest bodi know tellm model engin spec year product car histori info funki look car mail thank


In [ ]:
# Generate a corpus of splitted texts
corpus_gen=[doc.split() for doc in stem_data]

### **Part 1: Word2vec**
5. Run the following code. What is it doing?

In [ ]:
# Initializing the semi-trained model
path = get_tmpfile("word2vec.model")

# Applying the word2vec vectorization in the corpus
model = Word2Vec(corpus_gen, size=100, min_count=566)
model.save("word2vec.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


*The 100 is size is the nodes in the hidden layer.*

### Size is how big the embedding is

6. Print the sorted features of the model using model.wv.vocab.keys(). Compare them with those from Exercise 3.

In [ ]:
# Sorting the features
w2v_features = model.wv.vocab.keys()
print(sorted(w2v_features))
print("Number of features: " + str(len(w2v_features)))

['abl', 'accept', 'access', 'act', 'action', 'activ', 'actual', 'addit', 'address', 'advanc', 'ago', 'agre', 'allow', 'american', 'anonym', 'answer', 'anybodi', 'appear', 'appl', 'appli', 'applic', 'appreci', 'apr', 'april', 'arab', 'area', 'argument', 'arm', 'armenian', 'articl', 'ask', 'assum', 'atheist', 'attack', 'author', 'avail', 'awai', 'bad', 'base', 'basic', 'believ', 'best', 'better', 'bibl', 'big', 'bike', 'bit', 'black', 'board', 'bodi', 'book', 'box', 'bui', 'build', 'busi', 'call', 'came', 'car', 'card', 'care', 'carri', 'case', 'caus', 'center', 'certain', 'certainli', 'chang', 'check', 'children', 'chip', 'christ', 'christian', 'church', 'citi', 'claim', 'clear', 'clinton', 'clipper', 'close', 'code', 'color', 'com', 'come', 'command', 'comment', 'commun', 'compani', 'compar', 'complet', 'concern', 'connect', 'consid', 'contact', 'contain', 'continu', 'control', 'convert', 'copi', 'correct', 'cost', 'countri', 'coupl', 'cours', 'cover', 'creat', 'crime', 'current', 'dai

We have more features than in Exercise 3 because we didn't specify the maximum

7. Show model.wv['car']. What is it doing? Why does it have this size?

In [ ]:
#This is showing the vectors of car 
model.wv['car']

array([-0.14637282,  0.98556244,  1.6013836 , -0.03132056, -0.5604003 ,
       -0.81393015, -1.30699   ,  0.21027012,  1.0483739 , -0.9911072 ,
       -0.05815872, -0.8272026 ,  1.2839276 ,  0.45224813, -0.103787  ,
        0.2672654 , -0.2948971 , -0.14477168, -1.1630518 , -0.17413844,
        1.0117905 ,  0.25145894,  1.2817863 ,  0.4804006 , -1.0387725 ,
       -2.2523549 ,  0.24814583,  1.8395394 , -0.08427715, -0.24417557,
        0.9415932 , -0.6933158 , -0.15570147,  1.0521992 ,  0.2688034 ,
        1.4255916 , -0.4079378 ,  0.11174265,  0.40249506,  0.6500152 ,
       -0.20325631,  1.0814307 ,  0.24425259,  0.12754859, -0.17260498,
       -0.65929645,  0.8306198 , -0.444138  , -1.4689739 , -1.2078575 ,
       -0.6934617 ,  1.1964586 , -2.0705087 ,  1.5931517 , -2.300306  ,
       -0.58168894,  0.39446354,  0.8557328 ,  0.20158881, -1.6523231 ,
        1.2672195 ,  1.0935109 , -1.1118063 ,  1.2197587 , -0.31890464,
        0.5593808 , -0.28223276, -1.071275  ,  0.31821355, -1.03

### It has that size because we specify it when initializing the model

8. Run model.wv.most_similar('car'). Does the result make sense?

In [ ]:
# With this we get the similar words of car
model.wv.most_similar('car')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('bike', 0.6594415903091431),
 ('bui', 0.5785438418388367),
 ('nice', 0.5288523435592651),
 ('road', 0.511917769908905),
 ('fast', 0.4591946303844452),
 ('price', 0.45437362790107727),
 ('friend', 0.45134100317955017),
 ('pai', 0.4461021423339844),
 ('speed', 0.43983107805252075),
 ('sell', 0.43869104981422424)]

*Is there a threshold?? of a very similar cosine distance?*

### The result make sense to some extend, showing us bike, speed and road as similar to car

9. Run model.wv.most_similar(positive=['bike', 'engin'], topn=1). What is it doing? Test other combinations.


In [ ]:
#This is getting most similar word to bike and engin, like a synonym
model.wv.most_similar(positive=['bike', 'engin'], topn=1)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('car', 0.6377220153808594)]

In [ ]:
#This is getting most similar word to attack and bad
model.wv.most_similar(positive=['attack', 'bad'], topn=1)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('stop', 0.7145868539810181)]

In [ ]:
#This is getting less similar word to attack and bad, like antonym
model.wv.most_similar(negative=['attack', 'bad'], topn=1)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('info', 0.5327364206314087)]

In [ ]:
#This is getting less similar word to correct
model.wv.most_similar(positive=['correct'], topn=1)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('error', 0.6655834317207336)]

It works quite good

10. Generate a corpus as a list, where for each document the list entry consists of the list of word2vec embeddings of the words in this document. 

> Remove from it the empty documents. Why were those generated?

In [ ]:
corpus_lst = []
empty_docs = []
index_list = []

for d in range(len(corpus_gen)):  #d as in doc
  embd_lst = []  #list of embeddings
  
  for w in corpus_gen[d]:   #w as in word
    try:
      embd_lst.append(model.wv[w])
    except:
      pass

  #checking if doc is empty  
  if len(embd_lst) != 0:
    corpus_lst.append(embd_lst)
    index_list.append(d)
  else:
    #to have the index of empty docs
    empty_docs.append(d)


In [ ]:
print(empty_docs)

[800, 1015, 1410, 1636, 3682, 4474, 4829, 6294, 6303, 6324, 7709, 8076, 9622, 9989]


In [ ]:
len(corpus_lst)

11300


> Aggregate the values at a document level (i.e., one embedding per document) by averaging the word embeddings. 

In [ ]:
avg_corpus_lst = [sum(d) / len(d) for d in corpus_lst]
print(len(avg_corpus_lst))

11300



> Store the result for all documents in a data frame.

In [ ]:
#using index_list to keep the original index from the corpus
w2v_df = pd.DataFrame(avg_corpus_lst, index=index_list)
w2v_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.115746,0.346055,0.491456,0.121413,0.079659,-0.116975,-0.174135,-0.119210,0.371539,-0.150262,-0.186835,-0.564306,0.392212,0.178459,-0.043869,0.044449,0.035643,0.248571,-0.375743,0.064097,0.114127,0.281326,0.453258,0.196232,-0.148341,-0.436081,-0.092475,0.664568,-0.140854,0.006391,0.346249,-0.198806,0.105210,0.565256,0.357739,0.435850,0.064172,-0.166957,0.092945,0.179849,...,0.265036,0.407790,-0.144942,0.370585,-0.149648,0.104838,-0.401898,-0.284797,0.242510,-0.415179,-0.142128,0.204276,0.112435,0.080985,-0.546583,0.015049,-0.074467,-0.291051,-0.161736,0.105428,-0.022917,-0.242474,-0.378325,0.309111,0.000134,-0.420909,0.162460,0.614611,0.185027,0.335989,-0.454446,-0.094241,0.098293,0.450260,-0.000921,0.055754,0.350806,0.081844,-0.007381,-0.058043
1,0.194835,0.044899,0.040681,-0.060655,0.017288,0.054058,0.146961,0.180149,0.010209,0.157936,-0.064834,-0.207949,0.095307,0.464591,0.235780,-0.384986,0.186484,0.026037,0.293431,0.183401,-0.030151,0.304702,0.074193,0.091067,-0.259002,0.248643,-0.436143,-0.122579,-0.337997,-0.064507,-0.172225,-0.264132,-0.033212,0.160498,0.259907,0.191276,0.136686,0.131775,0.287449,0.312058,...,0.284999,0.014143,0.079842,0.401564,0.032416,0.101119,0.037380,-0.072989,0.021628,-0.125201,-0.170204,0.198925,-0.012987,0.339347,-0.586944,0.184637,0.041451,-0.016188,-0.001760,-0.001433,0.155419,0.353284,-0.237219,-0.001307,-0.270063,0.157930,0.421950,-0.058791,-0.101004,0.109225,-0.406577,0.142414,0.033228,0.060010,0.171585,0.146355,-0.187961,-0.084116,-0.019380,0.085351
2,0.075178,-0.033329,0.172677,0.187888,0.060043,0.051471,0.246902,-0.015402,0.128208,-0.104511,-0.075165,-0.107310,-0.209495,0.186609,0.152026,-0.219980,-0.089289,0.089099,-0.066401,0.118157,0.009739,0.272556,0.127842,0.129828,-0.213406,0.100345,-0.318316,0.066507,0.003315,0.207295,-0.053648,-0.125496,0.194709,0.310627,0.042245,-0.247857,-0.028095,-0.110320,0.149030,0.129186,...,0.202695,0.019816,-0.021175,0.153387,0.188063,-0.055097,-0.413697,-0.099593,-0.003911,-0.176802,-0.062280,0.315230,-0.099425,0.136124,-0.298901,-0.012298,-0.082239,0.036413,-0.166253,0.133600,0.051405,-0.031441,-0.115950,0.267420,-0.270000,0.086568,0.064360,0.100947,0.050668,0.037131,-0.248791,0.025441,0.080475,0.255783,0.180637,-0.068833,0.268717,0.002451,-0.052029,0.015871
3,-0.077582,0.145228,-0.097053,-0.008064,0.236674,0.174005,0.366116,0.311394,0.175824,0.083834,-0.066011,-0.072930,-0.341688,0.271677,0.404952,-0.589545,0.218617,-0.033446,-0.150516,0.067210,0.116956,0.584135,-0.070125,0.240222,0.188668,-0.058251,-0.244791,0.337257,0.028926,-0.032046,-0.407054,-0.079370,-0.050515,-0.084721,0.333295,-0.047619,0.099733,-0.008626,0.381635,-0.131042,...,0.088898,0.026447,-0.048576,0.105267,0.212922,0.079488,-0.126846,0.169406,-0.066401,-0.114051,-0.190268,-0.066663,-0.001433,0.117705,-0.288064,-0.219889,0.207585,0.099548,-0.034949,0.001419,0.035105,-0.209180,-0.060104,-0.142465,-0.102963,0.154876,0.367982,0.297486,0.168493,0.255885,-0.028961,-0.062082,-0.173889,0.056714,-0.227807,-0.075816,-0.125499,-0.147846,-0.240588,-0.079845
4,0.008898,0.240865,0.062242,-0.082950,0.127834,0.143931,0.327371,0.282790,0.175498,-0.220475,-0.218737,0.043127,-0.448824,0.182799,0.406757,-0.271944,-0.180451,-0.146458,-0.016162,-0.125024,0.003330,0.286768,0.121201,0.041618,-0.058492,0.166994,-0.355898,-0.208345,0.270745,0.059024,-0.190921,-0.127404,0.006508,-0.278231,-0.004818,-0.434680,-0.037802,0.063300,0.327257,0.193587,...,0.137147,-0.054372,0.066112,0.103271,0.515827,0.114029,-0.101729,0.043100,0.182746,-0.044693,0.035732,0.134505,-0.199799,0.021799,-0.127459,0.088038,0.045424,0.140401,-0.077744,0.053173,-0.088336,-0.069454,-0.238761,-0.316477,0.016533,0.313346,0.400096,0.137363,-0.203706,-0.084819,-0.128551,-0.149213,-0.229851,0.123029,0.077479,-0.1539

11. Find the most similar words to the embedding representing the first document from 10. 


> Also cosine() to determine the document on the corpus most similar to it.



In [ ]:
#it gets the similarity by vector that has to be in a numpy array
model.wv.similar_by_vector(w2v_df.iloc[0,:].to_numpy(), topn=5)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('car', 0.8723206520080566),
 ('nice', 0.5938571691513062),
 ('bike', 0.5845675468444824),
 ('bui', 0.5576478242874146),
 ('look', 0.5467740297317505)]

In [ ]:
#cosine(w2v_df[:1])
similar_docs = cdist(w2v_df[:1], w2v_df, metric='cosine')
print(similar_docs)

[[0.         0.65077947 0.50877886 ... 0.86396651 1.09012187 0.46563418]]


In [ ]:
import numpy as np
print(np.argmax(similar_docs))

6490


In [ ]:
most_similar = w2v_df.index[np.argmax(similar_docs)]
print(most_similar)
print(stem_data[most_similar])

6500
inimit rushdi articl dbstu dbstu benedikt rosenau write assum religi law follow religion beg question religion right defin follow offend disagre religi law appli categor belong religion event judg appli prevent situat member religion charg claim member religion free happen gregg


### Part 2: Doc2vec
12. Run gensim doc2vec model on corpus_gen (vector_size=100, min_count=566). (Note: Doc2vec requires tagged documents). 

Determine the embedding representation for the first document as well as the document in the dataset most similar to it. 

Compare it to the result in 11. 

In [ ]:
 docs = [TaggedDocument(doc, [i]) for i, doc in enumerate(corpus_gen)]
 model_d2v = Doc2Vec(docs, vector_size=100, min_count=566)

In [ ]:
model_d2v[0] 

array([-0.00769571,  0.02575505,  0.02223225,  0.01527679, -0.01356297,
        0.00489009,  0.02474311, -0.03643313,  0.04081266, -0.02772816,
        0.02970245, -0.05956564, -0.0211523 ,  0.03362303,  0.04612701,
       -0.04549494,  0.02226132,  0.01454111, -0.03628159, -0.0172617 ,
       -0.01569478,  0.01674451,  0.02569687,  0.01674143,  0.04677236,
        0.07242755, -0.02726835,  0.06624119, -0.03433381,  0.05977648,
        0.01317966,  0.00062943,  0.00960542,  0.02635494,  0.05932525,
        0.01350342,  0.00621799, -0.05047704,  0.00494697,  0.02962803,
       -0.00451402, -0.04295325,  0.08832821,  0.09198774,  0.04377612,
       -0.01548638,  0.0547283 , -0.01762848, -0.05057545, -0.01462605,
        0.00014163,  0.00074124, -0.03490152,  0.04129788, -0.03654723,
       -0.02234605,  0.09587798,  0.05830868, -0.01738056, -0.02965097,
       -0.0107477 ,  0.04497408,  0.01547105,  0.08796446, -0.01800635,
        0.04384307, -0.0540631 ,  0.02155822,  0.04487549,  0.00

In [ ]:
most_similar2 = model_d2v.docvecs.most_similar(0, topn=1)
print(most_similar2)

[(958, 0.8973060250282288)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
print(stem_data[most_similar2[0][0]])

car articl apr wam umd edu lerxst wam umd edu thing write wonder enlighten car saw dai door sport car look late earli call bricklin door small addit bumper separ rest bodi know tellm model engin spec year product car histori info funki look car mail bricklin manufactur engin ford odd look encas bumper aren lot hem motor new usus list basic perform ford new style slap brought neighborhood lerxst rush fan robert seymour rseymour reed edu physic philosophi reed colleg nextmail accept artifici life project reed colleg reed solar energi project soltrain portland



Generate the final corpus as a data frame.


In [ ]:
d2v_df = pd.DataFrame()
for i in range(len(model_d2v.docvecs)):
  d2v_df = d2v_df.append(pd.Series(model_d2v.docvecs[1]), ignore_index=True)

In [ ]:
d2v_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.014268,-0.020844,0.015708,-0.015266,0.013023,0.011935,0.029978,0.023444,0.008436,0.005959,-0.001054,0.000272,0.033009,0.02525,0.010799,-0.026137,0.008986,-0.008573,0.055154,0.00482,-0.019473,0.030629,0.007406,-0.013729,-0.015364,0.014051,-0.056472,-0.039674,-0.056238,0.001247,-0.004781,-0.034979,-0.016578,0.039082,-0.00425,0.03612,-0.018737,0.015872,0.008743,-0.010307,...,0.042746,0.012748,0.005035,0.0183,-0.025726,-0.013014,0.013489,0.010146,-0.005598,-0.004773,-0.008157,0.015173,-0.012491,0.026006,-0.048524,0.030665,0.013788,-0.01577,0.009276,0.006422,0.017188,0.033319,-0.012391,-0.020306,-0.019503,-0.006663,0.0281,-0.015506,-0.009367,0.022402,0.003736,0.00926,0.000411,-0.008099,0.024666,0.026176,0.006419,-0.014455,-0.02141,0.013611
1,0.014268,-0.020844,0.015708,-0.015266,0.013023,0.011935,0.029978,0.023444,0.008436,0.005959,-0.001054,0.000272,0.033009,0.02525,0.010799,-0.026137,0.008986,-0.008573,0.055154,0.00482,-0.019473,0.030629,0.007406,-0.013729,-0.015364,0.014051,-0.056472,-0.039674,-0.056238,0.001247,-0.004781,-0.034979,-0.016578,0.039082,-0.00425,0.03612,-0.018737,0.015872,0.008743,-0.010307,...,0.042746,0.012748,0.005035,0.0183,-0.025726,-0.013014,0.013489,0.010146,-0.005598,-0.004773,-0.008157,0.015173,-0.012491,0.026006,-0.048524,0.030665,0.013788,-0.01577,0.009276,0.006422,0.017188,0.033319,-0.012391,-0.020306,-0.019503,-0.006663,0.0281,-0.015506,-0.009367,0.022402,0.003736,0.00926,0.000411,-0.008099,0.024666,0.026176,0.006419,-0.014455,-0.02141,0.013611
2,0.014268,-0.020844,0.015708,-0.015266,0.013023,0.011935,0.029978,0.023444,0.008436,0.005959,-0.001054,0.000272,0.033009,0.02525,0.010799,-0.026137,0.008986,-0.008573,0.055154,0.00482,-0.019473,0.030629,0.007406,-0.013729,-0.015364,0.014051,-0.056472,-0.039674,-0.056238,0.001247,-0.004781,-0.034979,-0.016578,0.039082,-0.00425,0.03612,-0.018737,0.015872,0.008743,-0.010307,...,0.042746,0.012748,0.005035,0.0183,-0.025726,-0.013014,0.013489,0.010146,-0.005598,-0.004773,-0.008157,0.015173,-0.012491,0.026006,-0.048524,0.030665,0.013788,-0.01577,0.009276,0.006422,0.017188,0.033319,-0.012391,-0.020306,-0.019503,-0.006663,0.0281,-0.015506,-0.009367,0.022402,0.003736,0.00926,0.000411,-0.008099,0.024666,0.026176,0.006419,-0.014455,-0.02141,0.013611
3,0.014268,-0.020844,0.015708,-0.015266,0.013023,0.011935,0.029978,0.023444,0.008436,0.005959,-0.001054,0.000272,0.033009,0.02525,0.010799,-0.026137,0.008986,-0.008573,0.055154,0.00482,-0.019473,0.030629,0.007406,-0.013729,-0.015364,0.014051,-0.056472,-0.039674,-0.056238,0.001247,-0.004781,-0.034979,-0.016578,0.039082,-0.00425,0.03612,-0.018737,0.015872,0.008743,-0.010307,...,0.042746,0.012748,0.005035,0.0183,-0.025726,-0.013014,0.013489,0.010146,-0.005598,-0.004773,-0.008157,0.015173,-0.012491,0.026006,-0.048524,0.030665,0.013788,-0.01577,0.009276,0.006422,0.017188,0.033319,-0.012391,-0.020306,-0.019503,-0.006663,0.0281,-0.015506,-0.009367,0.022402,0.003736,0.00926,0.000411,-0.008099,0.024666,0.026176,0.006419,-0.014455,-0.02141,0.013611
4,0.014268,-0.020844,0.015708,-0.015266,0.013023,0.011935,0.029978,0.023444,0.008436,0.005959,-0.001054,0.000272,0.033009,0.02525,0.010799,-0.026137,0.008986,-0.008573,0.055154,0.00482,-0.019473,0.030629,0.007406,-0.013729,-0.015364,0.014051,-0.056472,-0.039674,-0.056238,0.001247,-0.004781,-0.034979,-0.016578,0.039082,-0.00425,0.03612,-0.018737,0.015872,0.008743,-0.010307,...,0.042746,0.012748,0.005035,0.0183,-0.025726,-0.013014,0.013489,0.010146,-0.005598,-0.004773,-0.008157,0.015173,-0.012491,0.026006,-0.048524,0.030665,0.013788,-0.01577,0.009276,0.006422,0.017188,0.033319,-0.012391,-0.020306,-0.019503,-0.006663,0.0281,-0.015506,-0.009367,0.022402,0.003736,0.00926,0.000411,-0.008099,0.024666,0.026176,0.006419,-0.014455,-0.02141,0.013611


### Part 3: ELMo
13. Run the following code. What is it doing?

In [ ]:
elmo = hub.Module("https://tfhub.dev/google/elmo/3")

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/elmo/3'.
INFO:absl:Downloaded https://tfhub.dev/google/elmo/3, Total size: 357.40MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/elmo/3'.


In [ ]:
embeddings = elmo(stem_data[0:10], signature="default",as_dict=True)["elmo"]
#embeddings = elmo(stem_data[0:10], signature="default",as_dict=True)["default"]
embeddings.shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


TensorShape([10, 211, 1024])

In [ ]:
#init = tf.compat.v1.initialize_all_variables()

with tf.compat.v1.Session() as sess: 
  sess.run(tf.compat.v1.global_variables_initializer())
  corpus_elm = sess.run(embeddings)

*Returns an Op that initializes global variables.* --> tf.compat.v1.global_variables_initializer()

In [ ]:
print(corpus_elm.shape)

(10, 1024)


a) Print the embedding values for the first document. Can you interpret it?

In [ ]:
corpus_elm[0]

array([ 0.07138148,  0.19519429, -0.09149865, ..., -0.37476006,
        0.18867464,  0.16174582], dtype=float32)

b) Store the embeddings for all documents in a data frame corpus_elm_df. How can
you use corpus_elm_df for further tasks?

Note: you can use more than 10 documents, but make sure you distribute those in
different sessions.

In [ ]:
corpus_elm_df = pd.DataFrame(corpus_elm)
corpus_elm_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.071381,0.195194,-0.091499,-0.056139,0.060336,0.602086,-0.428267,0.147217,-0.223365,-0.002364,-0.360357,0.267460,-0.653219,0.014107,-0.011198,0.153389,0.378626,0.386909,-0.418051,0.069700,0.755093,-0.149906,0.002108,-0.310571,0.213514,-0.101479,0.197859,-0.121517,0.207179,0.278058,0.341266,0.185140,0.188046,-0.467464,0.003810,0.232802,-0.347815,0.272836,0.354792,0.684967,...,0.354261,0.281943,0.001335,0.397741,0.297757,-0.502097,0.096041,0.063876,0.032441,-0.173737,0.188458,0.047619,-0.381535,-0.310682,-0.101096,0.348021,0.140095,-0.000057,-0.233020,0.182884,0.396763,-0.139555,0.102448,-0.637644,-0.141189,-0.086752,-0.182303,0.053941,-0.247799,-0.055875,-0.276381,0.332053,-0.225546,0.123052,-0.108826,-0.335023,0.275925,-0.374760,0.188675,0.161746
1,0.197677,0.139034,-0.162011,-0.094836,0.130264,0.229831,-0.285739,0.421668,0.197578,-0.164257,-0.247899,0.047955,-0.567297,0.121708,0.165591,0.036047,0.281482,0.446328,-0.587396,-0.068240,0.624128,0.427305,-0.050798,-0.060474,0.200221,0.143442,0.347401,-0.088222,0.098820,0.304108,-0.104149,0.164165,0.228108,-0.653909,0.032777,0.108331,-0.268706,0.167775,0.322713,0.587584,...,0.262288,0.285806,0.228223,0.475427,0.499776,-0.465738,0.265414,0.420376,0.114693,-0.120252,-0.058504,-0.070585,-0.316707,-0.141274,0.093630,0.424727,0.224331,0.442175,0.111860,0.140983,0.555419,-0.242645,0.113901,-0.303311,-0.075745,-0.244403,0.044823,-0.262665,-0.206717,0.025739,-0.278786,-0.067715,0.270636,0.083944,0.291727,-0.105083,0.131594,-0.084821,0.454332,-0.033999
2,0.170616,-0.435246,0.100215,-0.043610,0.330289,0.502139,-0.226096,0.234854,-0.316113,-0.091387,-0.271925,0.111460,-0.611972,-0.076087,0.107744,-0.005868,0.365481,0.176191,-0.519185,0.150881,0.567667,-0.049746,0.220452,-0.135826,0.338678,-0.064376,0.058292,0.146376,0.074083,0.461452,0.372899,0.268020,0.067177,-0.304049,-0.070484,0.348015,-0.228523,0.041331,0.205810,0.440166,...,0.326271,0.417339,0.174676,0.404286,0.205106,-0.412850,0.197703,0.127029,0.210548,-0.287393,-0.018736,-0.104600,-0.209796,-0.345535,0.006915,0.646273,-0.235767,0.334497,0.054455,0.047398,0.297288,-0.118444,0.037223,-0.323689,-0.179704,-0.209463,-0.090724,-0.256350,-0.182799,-0.287376,-0.294561,0.072671,0.127512,0.115997,0.360114,-0.090129,0.050056,-0.022320,0.432490,-0.021742
3,0.112675,-0.106532,0.029696,-0.188042,0.024403,0.845828,-0.371575,0.434558,-0.001990,0.009475,-0.358585,0.081786,-0.709006,-0.042012,-0.072861,0.049609,0.480018,0.533367,-0.708592,0.338966,0.901059,-0.188399,0.295823,-0.069612,0.442792,-0.050616,-0.087286,0.070540,0.349901,0.720822,0.360695,0.262907,0.411075,-0.229887,-0.177007,0.218887,-0.321949,0.203934,0.382982,0.505422,...,0.347870,0.343024,0.249943,0.487751,0.137853,-0.527354,0.218220,0.322993,0.067699,-0.305773,-0.112115,-0.215133,-0.114937,-0.325125,-0.037927,0.538973,-0.173471,0.268805,0.058968,0.047156,0.247988,-0.069754,0.129090,-0.370793,-0.252172,0.019999,0.002496,-0.264802,0.032591,-0.134799,-0.307827,0.178420,0.267531,0.197801,0.301834,0.044331,-0.034822,-0.060457,0.317629,0.018689
4,0.114391,-0.255723,-0.013560,-0.353646,0.287903,0.444778,-0.394945,0.236275,-0.257917,-0.194771,-0.283504,-0.182900,-0.664141,0.246531,0.118883,0.154575,0.502999,0.335899,-0.479125,0.206198,0.599743,0.122603,0.152474,-0.028882,0.567735,-0.132621,0.008428,0.004917,0.321629,0.602193,0.201842,0.336668,0.075853,-0.191283,-0.038348,0.239463,-0.378110,0.131521,0.214341,0.436583,...,0.550671,0.641285,0.642541,0.608518,0.576593,-0.262087,0.270709,-0.085255,0.284999,0.048246,-0.287580,-0.335839,-0.216407,-0.075507,0.087952,0.153030,-0.057662,0.266731,0.244756,-0.019399,0.347825,-0.268087,-0.268204,-0.268714,-0.482284,-0.037457,-0.146996,-0.309631,-0.240904,-0.1989

Maybe we could use the elmo embeddings to check for similarity?

In [ ]:
#cosine(w2v_df[:1])
elmo_similar = cdist(corpus_elm_df[:1], corpus_elm_df, metric='cosine')
print(elmo_similar)

[[1.11022302e-16 1.94381002e-01 2.19225268e-01 2.12467655e-01
  3.00232087e-01 2.28327178e-01 3.34123060e-01 2.12331353e-01
  2.99280351e-01 1.61800893e-01]]


In [ ]:
elmo_most_sim = corpus_elm_df.index[np.argmax(elmo_similar)]
print(elmo_most_sim)
print(stem_data[elmo_most_sim])

6
brain tumor treatment thank peopl respond request info treatment astrocytoma email couldn thank directli mail bounc prob sean debra sharon thought publicli thank thank sure glad accident hit instead try delet file septemb hmmm new


In [ ]:
print(stem_data[corpus_elm_df[:1].index[0]])

car wonder enlighten car saw dai door sport car look late earli call bricklin door small addit bumper separ rest bodi know tellm model engin spec year product car histori info funki look car mail thank


### Part 4: BERT
14. Load and import the lemmatized dataset from Exercise 2. Call it data_lemma.

In [ ]:
uploaded = files.upload()

Saving Lemma.pkl to Lemma.pkl


In [ ]:
# Load dataset and check first entry
data_lemma = pickle.load(open('Lemma.pkl','rb'))
print(data_lemma[0])

car wonder enlighten car see day door sport car look late early call bricklin door small addition bumper separate rest body know tellme model engine specs year production car history info funky look car mail thank


15. Run the following code and print the output of each step for the first document. What
are those steps doing and why? Hint: You may wish to do a web research to find out.



> To make this pooling scheme work, BERT prepends a **[CLS]** token (short for "classification") to the start of each sentence (this is essentially like a start-of-sentence token).



> When taking two sentences as input, BERT separates the sentences with a special **[SEP]** token. 




In [ ]:
sentences = ["[CLS] " + query + " [SEP]" for query in data_lemma]
print(sentences[0])

[CLS] car wonder enlighten car see day door sport car look late early call bricklin door small addition bumper separate rest body know tellme model engine specs year production car history info funky look car mail thank [SEP]


> Our first step will be initializing a tokenizer so that we handle the input in the same way the BERT model did while pretraining.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print(tokenized_texts[0])


['[CLS]', 'car', 'wonder', 'en', '##light', '##en', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'brick', '##lin', 'door', 'small', 'addition', 'bumper', 'separate', 'rest', 'body', 'know', 'tell', '##me', 'model', 'engine', 'spec', '##s', 'year', 'production', 'car', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', '[SEP]']


> pad_sequences is used to ensure that all sequences in a list have the same length. By default this is done by padding 0 in the beginning of each sequence until each sequence has the same length as the longest sequence

In [ ]:
sentences_padded = pad_sequences(tokenized_texts, dtype=object,maxlen= 200, value='[PAD]', truncating="post",padding="post")
print(sentences_padded[0])

['[CLS]' 'car' 'wonder' 'en' '##light' '##en' 'car' 'see' 'day' 'door'
 'sport' 'car' 'look' 'late' 'early' 'call' 'brick' '##lin' 'door' 'small'
 'addition' 'bumper' 'separate' 'rest' 'body' 'know' 'tell' '##me' 'model'
 'engine' 'spec' '##s' 'year' 'production' 'car' 'history' 'info' 'funky'
 'look' 'car' 'mail' 'thank' '[SEP]' '[PAD]' '[PAD]' '[PAD]' '[PAD]'
 '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]'
 '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]'
 '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]'
 '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]'
 '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]'
 '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]'
 '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]'
 '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]'
 '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[P

In [ ]:
sentences_converted = [tokenizer.convert_tokens_to_ids(s) for s in sentences_padded]
print(sentences_converted[0])

[101, 2482, 4687, 4372, 7138, 2368, 2482, 2156, 2154, 2341, 4368, 2482, 2298, 2397, 2220, 2655, 5318, 4115, 2341, 2235, 2804, 21519, 3584, 2717, 2303, 2113, 2425, 4168, 2944, 3194, 28699, 2015, 2095, 2537, 2482, 2381, 18558, 24151, 2298, 2482, 5653, 4067, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
model= TFBertModel.from_pretrained('bert-base-uncased') 
input_ids=tf.cast(sentences_converted[0:10],tf.int32) 
outputs = model(input_ids)
last_hidden_state = outputs[0]
print(last_hidden_state)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


Tensor("tf_bert_model_2/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0", shape=(10, 200, 768), dtype=float32)


In [ ]:
mean_embeddings=tf.reduce_mean(last_hidden_state,1)
print(mean_embeddings)

Tensor("Mean:0", shape=(10, 768), dtype=float32)


In [ ]:
init = tf.compat.v1.global_variables_initializer()
with tf.compat.v1.Session() as sess: 
  sess.run(init)
  result = sess.run(mean_embeddings)


Store result in a data frame and print its head. What does this object contain? How can you use it for further tasks?
Note: you can use more than 10 documents, but make sure you distribute those in different sessions.

In [ ]:
corpus_bert_df = pd.DataFrame(result)
corpus_bert_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.170458,-0.653127,0.345098,-0.064987,-1.452454,-0.168962,0.686052,0.379111,0.522723,-0.053348,0.652380,0.587839,1.122612,-1.880497,-0.793651,-0.019234,1.056585,0.004015,1.315714,0.383291,1.209543,0.321897,-0.434226,-0.589653,-1.053622,-0.732756,0.297357,-0.359043,-0.410570,1.480325,1.040380,0.180598,1.147713,0.073048,0.932117,-1.508389,1.130614,1.203676,-1.490379,-0.590931,...,0.196813,0.475859,-0.163888,0.232128,0.266089,0.889039,0.132070,0.617506,1.787864,-0.517471,0.812471,1.053689,-0.257285,-0.217252,1.158489,-0.411412,0.358174,-0.040617,-0.999985,-0.812283,0.751829,-0.306615,-0.844451,0.285206,0.981825,-0.227392,-0.505802,0.280369,1.311274,0.738192,1.224730,-0.475266,-1.291861,0.081470,0.534557,-0.354931,0.918021,0.954238,-0.846923,0.782408
1,-0.253974,-0.673361,0.511080,-0.050205,-1.532552,-0.155999,0.710507,0.464843,0.450612,-0.028470,0.506875,0.598392,1.029074,-1.684195,-0.690386,0.140580,0.902768,-0.088746,1.147090,0.275583,1.216975,0.413588,-0.249957,-0.664633,-1.025091,-0.571542,0.378328,-0.424285,-0.324310,1.452877,0.899751,0.113077,1.230055,0.080670,0.988014,-1.455190,1.177265,1.225021,-1.595695,-0.483198,...,0.175497,0.646176,-0.130074,0.337264,0.309437,0.732836,0.117325,0.734559,1.727379,-0.445629,0.847046,1.029743,-0.155353,-0.222819,1.188308,-0.345413,0.354500,-0.113541,-0.934370,-0.664493,0.665019,-0.317215,-0.637114,0.204578,0.941765,-0.258261,-0.429321,0.148350,1.242222,0.757759,1.131970,-0.749254,-1.133600,0.052506,0.367574,-0.403532,0.878676,1.017440,-0.801245,0.763973
2,-0.172993,-1.019660,0.756207,0.131656,-1.612421,-0.010500,0.630341,0.901812,0.484832,-0.122190,0.274887,0.397976,0.662662,-1.077864,-0.503960,0.466283,0.521020,-0.383303,0.468499,0.232360,1.001407,0.772604,0.096413,-0.714483,-0.825668,-0.100099,0.530355,-0.570946,-0.052786,1.082864,0.506244,0.001263,1.408715,0.034017,1.025525,-1.165022,1.063148,1.027993,-1.665717,-0.306985,...,0.163915,0.889299,0.073236,0.475210,0.375816,0.491722,0.102273,0.681327,1.241831,-0.430871,0.842376,0.969332,-0.085378,0.013679,1.174982,-0.320897,0.388424,-0.548179,-0.699639,-0.334304,0.495946,-0.381873,-0.079468,-0.078580,0.722576,-0.388645,-0.190262,0.110425,1.043377,0.782169,0.832200,-1.298404,-0.758087,0.000477,0.201861,-0.520458,0.764087,0.952692,-0.623989,0.690643
3,-0.243290,-0.724696,0.516203,0.089570,-1.462053,-0.095110,0.694003,0.542176,0.446446,-0.048586,0.567066,0.556603,0.913316,-1.592775,-0.724172,0.149579,0.836860,-0.124864,0.991511,0.248721,1.128001,0.449874,-0.232177,-0.619936,-0.926255,-0.468638,0.426633,-0.462715,-0.247197,1.380158,0.879754,0.151052,1.255262,0.119380,0.964753,-1.349832,1.132657,1.148606,-1.610573,-0.556562,...,0.162831,0.605357,-0.071513,0.396362,0.387047,0.743945,0.093980,0.705977,1.583067,-0.487066,0.834493,1.060038,-0.180655,-0.198952,1.251622,-0.367737,0.420035,-0.165058,-1.004032,-0.604056,0.663242,-0.352892,-0.589845,0.157812,0.925024,-0.231996,-0.321238,0.188851,1.188400,0.755945,1.096868,-0.755400,-1.110425,0.072539,0.417367,-0.454594,0.930092,0.945061,-0.771752,0.713548
4,-0.193788,-0.891993,0.570361,0.038182,-1.505997,-0.168567,0.648142,0.597501,0.437459,-0.025764,0.455223,0.500430,0.960452,-1.500156,-0.731928,0.279215,0.701769,-0.220644,0.846265,0.287925,1.178856,0.547632,-0.261870,-0.590929,-0.958314,-0.391722,0.454164,-0.488057,-0.140341,1.345245,0.779324,0.099283,1.233431,0.166530,0.908874,-1.413623,1.154562,1.101865,-1.618606,-0.490500,...,0.193484,0.660452,-0.042251,0.306545,0.348327,0.738846,0.034003,0.704891,1.528556,-0.478298,0.815849,1.026460,-0.172018,-0.178020,1.233534,-0.340832,0.349809,-0.217813,-0.876745,-0.587159,0.566960,-0.341688,-0.502084,0.081650,0.892714,-0.250044,-0.317657,0.227114,1.178012,0.766620,1.044332,-0.877433,-0.94